# 2021 Train 2022 H1 Val H2 Test Time Forecasting 

In [ ]:
import os
import datetime
from random import seed
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras import layers
import keras
from keras import regularizers


In [ ]:
# Set Random Number Seed
keras.utils.set_random_seed(30)
tf.config.experimental.enable_op_determinism()

In [ ]:
df = pd.read_csv("2021-2022 data.csv")
df

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)


In [ ]:
n = len(df)
test = df[int(n*0.75):]
test_dates = test['datetime']
test_dates

In [ ]:
df = df[['Waterlevel_Sto_Nino', 'Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro']]
df

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
n = len(df)
train_df = df[0:int(n*0.5)]
val_df = df[int(n*0.5):int(n*0.75)]
test_df = df[int(n*0.75):]

num_features = df.shape[1]
num_features

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

    def plot(self, model=None, plot_col='Waterlevel_Sto_Nino', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col}')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                          marker='X', edgecolors='k', label='Predictions',
                          c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [h]')


# Dense Neural Network (DNN)

In [ ]:
wide_window = WindowGenerator(input_width = 6, label_width = 6, shift = 1, label_columns = ['Waterlevel_Sto_Nino'])
wide_window


### Hyperparameter tuning for the number of units
1. Number of dense layers - 2 to 5
2. Number of units per layer - 16 to 128 (step = 16)
3. Type of activation function - relu or tanh or sigmoid
4. Penalty of L2 Regularisation - lambda = 0.001, 0.0005, 0.0001

In [ ]:
dnn_model_names = []
dnn_list_of_models = []

In [ ]:
# Dense Neural Network
def dense_model(no_of_layers, no_of_units, acti, penalty):
    model_name = f'Dense_{no_of_layers}_{no_of_units}_{activation}_{penalty}'
    model = tf.keras.Sequential()
    for i in range(no_of_layers):
        model.add(layers.Dense(units = no_of_units, activation = acti, kernel_regularizer=regularizers.l2(penalty)))
    model.add(layers.Dense(units = 1))
    
    return model_name, model

for no_of_layers in range(2,6):
    for no_of_units in range(16,129,16):
        for activation in ['relu', 'tanh', 'sigmoid']:
            for penalty in [0.001, 0.0005, 0.0001]:
                model_name, model = dense_model(no_of_layers, no_of_units, activation, penalty)
                dnn_model_names.append(model_name)
                dnn_list_of_models.append(model)

In [ ]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanSquaredError(), r_square, NSE])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    return history

In [ ]:
performance = {}

In [ ]:
dense_val_performance = {}
dnn_history = {}
for count, model in enumerate(dnn_list_of_models):
    model_name = dnn_model_names[count]
    dnn_history[model_name] = compile_and_fit(model, wide_window)
    dense_val_performance[model_name] = model.evaluate(wide_window.val, return_dict = True)


In [ ]:
dense_val_performance

In [ ]:
dense_models = pd.DataFrame.from_dict(dense_val_performance)
dense_models = dense_models.transpose()
dense_models = dense_models.sort_values(by = "loss")
dense_models = dense_models.reset_index()
dense_models


In [ ]:
# We evaluate the best model according to least validation loss
# Constructing the best model
best_dnn_index = dense_models['loss'].idxmin()
best_dnn_model = dnn_list_of_models[best_dnn_index]
performance['Dense'] = best_dnn_model.evaluate(wide_window.test, verbose = 0, return_dict = True)

In [ ]:
performance

In [ ]:
# Test for Overfitting 
dnn_name = dnn_model_names[best_dnn_index]
dense_history = dnn_history[dnn_name].history


In [ ]:
from matplotlib import pyplot as plt
plt.plot(dense_history['loss'])
plt.plot(dense_history['val_loss'])
plt.title('DNN Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
dense_history

In [ ]:
dh = dense_history['loss']
dvh = dense_history['val_loss']

In [ ]:
dh = dh[1:]
dvh = dvh[1:]


In [ ]:
dnn_n = len(dh)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.linspace(1,dnn_n, dnn_n), dh)
plt.plot(np.linspace(1,dnn_n, dnn_n), dvh)
plt.title('DNN Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
dense_history_2 = compile_and_fit(best_dnn_model, wide_window)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(dense_history_2.history['loss'])
plt.plot(dense_history_2.history['val_loss'])
plt.title('DNN Model Loss Learning Curve (2nd Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
dense_predictions = best_dnn_model.predict(test_df)
actual = test_df['Waterlevel_Sto_Nino']
from matplotlib import pyplot as plt
import matplotlib.dates as mdates  # Import the dates module



In [ ]:
plt.figure(figsize=(14, 9))  # Adjust the width and height as needed
plt.plot(test_dates, dense_predictions, c = "blue", label = "predictions", alpha = 0.5)
plt.plot(test_dates, actual, c = "red", label = "actual", alpha = 0.5)
plt.title('Predictions vs Actual Values (DNN Test Set) - 2nd Half of 2022')
plt.ylabel('Water Level Sto Nino (meters)')
plt.xlabel('Datetime')

# Set x-axis major locator to show ticks for every month
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

# Format the dates as YYYY-MM-DD
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.gcf().autofmt_xdate()
plt.gca().xaxis_date()

plt.legend()
plt.show()

In [ ]:
# Set Random Number Seed
keras.utils.set_random_seed(30)
tf.config.experimental.enable_op_determinism()

In [ ]:
actual.idxmax()
actual[15193]
max(dense_predictions)

In [ ]:
cnn_model_names = []
cnn_list_of_models = []

In [ ]:
# Convolution Neural Network (CNN)
CONV_WIDTH = 6

conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=['Waterlevel_Sto_Nino'])

In [ ]:
# Convolution Neural Network (CNN)
def cnn_model(no_of_layers, no_of_units, acti, penalty):
    model_name = f'CNN_{no_of_layers}_{no_of_units}_{activation}_{penalty}'
    model = tf.keras.Sequential()
    model.add(layers.Conv1D(filters = 64, kernel_size =(CONV_WIDTH,), activation = acti, kernel_regularizer = regularizers.l2(penalty)))
    for i in range(no_of_layers):
        model.add(layers.Dense(units = no_of_units, activation = acti, kernel_regularizer = regularizers.l2(penalty)))
    model.add(layers.Dense(units = 1))
    
    return model_name, model

for no_of_layers in range(2,6):
    for no_of_units in range(16,129,16):
        for activation in ['relu', 'tanh', 'sigmoid']:
            for penalty in [0.001, 0.0005, 0.0001]:
                model_name, model = cnn_model(no_of_layers, no_of_units, activation, penalty)
                cnn_model_names.append(model_name)
                cnn_list_of_models.append(model)

In [ ]:
cnn_val_performance = {}
cnn_history = {}
for count, model in enumerate(cnn_list_of_models):
    model_name = cnn_model_names[count]
    cnn_history[model_name] = compile_and_fit(model, conv_window)
    cnn_val_performance[model_name] = model.evaluate(conv_window.val, return_dict = True)


In [ ]:
cnn_models = pd.DataFrame.from_dict(cnn_val_performance)
cnn_models = cnn_models.transpose()
cnn_models = cnn_models.sort_values(by = 'loss')
cnn_models = cnn_models.reset_index()
cnn_models

In [ ]:
# We get the best performing model that has the least validation loss
best_cnn_index = cnn_models['loss'].idxmin()
best_cnn_model = cnn_list_of_models[best_cnn_index]
performance['CNN'] = best_cnn_model.evaluate(conv_window.test, verbose = 0, return_dict = True)

In [ ]:
# Test for Overfitting CNN 
cnn_name = cnn_model_names[best_cnn_index]
conv_history = cnn_history[cnn_name].history


In [ ]:
performance

In [ ]:
from matplotlib import pyplot as plt
plt.plot(conv_history['loss'])
plt.plot(conv_history['val_loss'])
plt.title('CNN Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
ch = conv_history['loss']
cvh = conv_history['val_loss']
ch = ch[1:]
cvh = cvh[1:]


In [ ]:
cnn_n = len(ch)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.linspace(1,cnn_n, cnn_n), ch)
plt.plot(np.linspace(1,cnn_n, cnn_n), cvh)
plt.title('CNN Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
conv_history_2 = compile_and_fit(best_cnn_model, conv_window)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(conv_history_2.history['loss'])
plt.plot(conv_history_2.history['val_loss'])
plt.title('CNN Model Loss Learning Curve (2nd Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
prediction_data = test_df.values.reshape(1, -1, CONV_WIDTH)  # Add batch dimension of 1
conv_predictions = best_cnn_model.predict(prediction_data)
actual = test_df['Waterlevel_Sto_Nino']

In [ ]:
# reshape the conv_predictions array
# delete the first 5 values of the actual and test_dates arrays (corresponding to t = 0 to 4)

conv_predictions = conv_predictions.reshape(4375)
actual = actual[5:]
test_dates_cnn = test_dates[5:]


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.dates as mdates  # Import the dates module

plt.figure(figsize=(14, 9))  # Adjust the width and height as needed
plt.plot(test_dates_cnn, conv_predictions, c = "blue", label = "predictions", alpha = 0.5)
plt.plot(test_dates_cnn, actual, c = "red", label = "actual", alpha = 0.5)
plt.title('Predictions vs Actual Values (CNN Test Set) - 2nd Half of 2022')
plt.ylabel('Water Level Sto Nino (meters)')
plt.xlabel('Datetime')

# Set x-axis major locator to show ticks for every month
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

# Format the dates as YYYY-MM-DD
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.gcf().autofmt_xdate()
plt.gca().xaxis_date()

plt.legend()
plt.show()

In [ ]:
# Set Random Number Seed
keras.utils.set_random_seed(30)
tf.config.experimental.enable_op_determinism()

In [ ]:
lstm_model_names = []
lstm_list_of_models = []

In [ ]:
# LSTM / RNN

# Dense Neural Network
def lstm_model(no_of_layers, no_of_units, acti, penalty):
    model_name = f'LSTM_{no_of_layers}_{no_of_units}_{activation}_{penalty}'
    model = tf.keras.Sequential()
    model.add(layers.LSTM(64, return_sequences = True, kernel_regularizer = regularizers.l2(penalty)))
    for i in range(no_of_layers):
        model.add(layers.Dense(units = no_of_units, activation = acti, kernel_regularizer = regularizers.l2(penalty)))
    model.add(layers.Dense(units = 1))
    
    return model_name, model

for no_of_layers in range(2,6):
    for no_of_units in range(16,129,16):
        for activation in ['relu', 'tanh', 'sigmoid']:
            for penalty in [0.001, 0.0005, 0.0001]:
                model_name, model = lstm_model(no_of_layers, no_of_units, activation, penalty)
                lstm_model_names.append(model_name)
                lstm_list_of_models.append(model)

In [ ]:
lstm_val_performance = {}
lstm_history = {}
for count, model in enumerate(lstm_list_of_models):
    model_name = lstm_model_names[count]
    lstm_history[model_name] = compile_and_fit(model, wide_window)
    lstm_val_performance[model_name] = model.evaluate(wide_window.val, return_dict = True)
print(lstm_val_performance)

In [ ]:
lstm_models = pd.DataFrame.from_dict(lstm_val_performance)
lstm_models = lstm_models.transpose()
lstm_models = lstm_models.sort_values(by = "loss")
lstm_models = lstm_models.reset_index()
lstm_models

In [ ]:
# We evaluate the best model according to least validation loss
# Constructing the best model
best_lstm_index = lstm_models['loss'].idxmin()
best_lstm_model = lstm_list_of_models[best_lstm_index]
performance['LSTM'] = best_lstm_model.evaluate(wide_window.test, verbose = 0, return_dict = True)

In [ ]:
performance

In [ ]:
# Test for Overfitting LSTM 
lstm_name = lstm_model_names[best_lstm_index]
long_history = lstm_history[lstm_name].history
long_history

In [ ]:
from matplotlib import pyplot as plt
plt.plot(long_history['loss'])
plt.plot(long_history['val_loss'])
plt.title('LSTM Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
lh = long_history['loss']
lvh = long_history['val_loss']
lh = lh[1:]
lvh = lvh[1:]
lnn_n = len(lh)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.linspace(1,lnn_n, lnn_n), lh)
plt.plot(np.linspace(1,lnn_n, lnn_n), lvh)
plt.title('LSTM Model Loss Learning Curve (1st Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
long_history_2 = compile_and_fit(best_lstm_model, wide_window)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(long_history_2.history['loss'])
plt.plot(long_history_2.history['val_loss'])
plt.title('LSTM Model Loss Learning Curve (2nd Training Process)')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
prediction_data = test_df.values.reshape(1, -1, 6)  # Add batch dimension of 1
lstm_predictions = best_lstm_model.predict(prediction_data)
actual = test_df['Waterlevel_Sto_Nino']

In [ ]:
print(lstm_predictions.shape)
lstm_predictions = lstm_predictions.reshape(4380)
print(actual.shape)
print(lstm_predictions.shape)


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.dates as mdates  # Import the dates module

plt.figure(figsize=(14, 9))  # Adjust the width and height as needed
plt.plot(test_dates, lstm_predictions, c = "blue", label = "predictions", alpha = 0.5)
plt.plot(test_dates, actual, c = "red", label = "actual", alpha = 0.5)
plt.title('Predictions vs Actual Values (LSTM Test Set) - 2nd Half of 2022')
plt.ylabel('Water Level Sto Nino (meters)')
plt.xlabel('Datetime')

# Set x-axis major locator to show ticks for every month
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

# Format the dates as YYYY-MM-DD
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.gcf().autofmt_xdate()
plt.gca().xaxis_date()

plt.legend()
plt.show()

In [ ]:
performance

In [ ]:
max(lstm_predictions)

In [ ]:
pd.DataFrame.from_dict(performance).transpose()

## Predictions (starting from t = 5)

In [ ]:
test_dates_cnn


In [ ]:
print(dense_predictions.shape)
dense_predictions = dense_predictions.reshape(4380)

print(conv_predictions.shape)
print(lstm_predictions.shape)
dense_predictions
dense_predictions = dense_predictions[5:]
lstm_predictions = lstm_predictions[5:]

In [ ]:
actual = actual[5:]

In [ ]:
print(actual.shape)
print(dense_predictions.shape)
print(conv_predictions.shape)
print(lstm_predictions.shape)
print(test_dates_cnn.shape)

In [ ]:
data = {
    'datetime': test_dates_cnn.tolist(),
    'actual': actual.tolist(),
    'dense_predictions': dense_predictions.tolist(),
    'conv_predictions': conv_predictions.tolist(),
    'lstm_predictions': lstm_predictions.tolist()
}

predictions = pd.DataFrame(data)


In [ ]:
predictions

In [ ]:
predictions[predictions['actual'] == 18.5]

In [ ]:
predictions.to_csv('2H 2022 Predictions.csv')

## Multi-step models
- Uses the past 24 hours of data to predict the water level 24 hours into the future

In [ ]:
OUT_STEPS = 24
multistep_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

multistep_window

In [ ]:
multi_val_performance = {}
multi_performance = {}

In [ ]:
# Autoregressive model : decomposes the prediction into individual time steps, you use the output for the next prediction
# Autoregressive LSTM/RNN

class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(6)
    def warmup(self, inputs):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state = self.lstm_rnn(inputs)
        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state
    def call(self, inputs, training=None):
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Initialize the LSTM state.
        prediction, state = self.warmup(inputs)

        # Insert the first prediction.
        predictions.append(prediction)

        # Run the rest of the prediction steps.
        for n in range(1, self.out_steps):
            # Use the last prediction as input.
            x = prediction
            # Execute one lstm step.
            x, state = self.lstm_cell(x, states=state,
                                      training=training)
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)
            # Add the prediction to the output.
            predictions.append(prediction)

        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        print(predictions)
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        print(predictions)
        return predictions




In [ ]:
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)
prediction, state = feedback_model.warmup(multistep_window.example[0])
prediction.shape

In [ ]:
print('Output shape (batch, time, features): ', feedback_model(multistep_window.example[0]).shape)


In [ ]:
history = compile_and_fit(feedback_model, multistep_window)

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multistep_window.val, return_dict=True)
multi_performance['AR LSTM'] = feedback_model.evaluate(multistep_window.test, verbose=0, return_dict=True)

In [ ]:
multi_performance

In [ ]:
prediction_data = test_df.values.reshape(1, -1, 6)  # Add batch dimension of 1
ar_lstm_predictions = feedback_model.predict(prediction_data)
actual = test_df['Waterlevel_Sto_Nino']

In [ ]:
ar_lstm_predictions